<a href="https://colab.research.google.com/github/amine-76/Apprentissage-Automatique/blob/main/tp4_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : Analyse de spams avec des techniques de classification bayésienne naïve

Dans cet exercice, on va classer des emails comme spam ou non-spam selon une approche bayésienne naïve. L’ensemble des données est issu de 960 emails réels prétraités de la base Ling-spam (anciennement accessible à cette adresse http://csmining.org/index.php/ling-spam-datasets.html). Ces emails sont en langue anglaise.

## Données

Les données relatives à cet exercice sont à télécharger sur le cours Eureka sous forme d’un fichier compacté zip ’Données classification bayésienne’. L’ensemble des données se divise en 700 emails exemples d’apprentissage et 260 emails exemples de test, chaque sous-ensemble étant par moitié des spams et des non-spams.

Ces emails ont été pré-traités : élimination de mots non signifiants (the, and, ...) ; nombres et ponctuations supprimées ; regroupement de mots de même racine avec des terminaisons différentes (include, includes, included, ...) ; ...

Dans le fichier nommé `train-features.txt`, les données des emails exemples d’apprentissage sont présentées sous la forme :

    2 977 2
    2 1481 1
    2 1549 1
    
Chaque enregistrement (une ligne du fichier) comporte le numéro de l’email, le second nombre indique l’indice d’un mot dans le dictionnaire de mots utilisés pour l’analyse, et le troisième donne le nombre d’occurrences de ce mot dans l’email indiqué. Ici la première ligne précise que le mail numéro 2 a 2 occurrences du mot 977 du dictionnaire.

Il est à noter qu'il n'est pas utile de connaître les mots du dictionnaire dans le cadre de cet exercice.

Ce fichier va permettre de créer une matrice creuse (cf.https://fr.wikipedia.org/wiki/Matrice_creuse) qui va nous permettre de calculer nos statistiques plus facilement :

    numTrainDocs = 700 # nombre total d'emails exemples d’apprentissage
    numTokens = 2500 # nombre de mots dans le dictionnaire
    M = np.loadtxt(’train-features.txt’)


In [ ]:
import numpy as np

numTrainDocs = 700 # nombre total d'emails exemples d’apprentissage
numTokens = 2500 # nombre de mots dans le dictionnaire
M = np.loadtxt('train-features.txt')
print("M :\n",M)

M :
 [[1.000e+00 1.900e+01 2.000e+00]
 [1.000e+00 4.500e+01 1.000e+00]
 [1.000e+00 5.000e+01 1.000e+00]
 ...
 [7.000e+02 2.479e+03 2.000e+00]
 [7.000e+02 2.481e+03 2.000e+00]
 [7.000e+02 2.500e+03 3.000e+00]]


Puis créer la matrice creuse `train_matrix` de taille (700×2500) selon la configuration suivante :
`train_matrix[i,j]` correspond au nombre d'occurence du mot $j$ du dictionnaire dans l'email $i$.

Pour finir sur ces données, il manque encore les labels associés à ces emails, qui précisent si ce sont des spams (valeur 1) ou non (valeur 0) :

    train_labels = np.loadtxt(’train-labels.txt’)

Cela donne un vecteur de taille(m,1) où m est le nombre d'emails exemples (c'est à dire m = `numTrainDocs`)

In [ ]:
# Création de la matrice creuse
train_matrix = np.zeros((numTrainDocs, numTokens))
for i in range(M.shape[0]):
    train_matrix[int(M[i,0]-1), int(M[i,1]-1)] = M[i,2]
print("train_matrix :\n",train_matrix)
train_labels = np.loadtxt('train-labels.txt')
print("train_labels :\n",train_labels)


train_matrix :
 [[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 1.  0.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ...  0.  0.  0.]
 [ 3.  2. 17. ...  0.  0.  3.]]
train_labels :
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

## Méthode de classification bayésienne naïve
La méthode utilisée ici est décrite dans [1].

L'objectif est de classifier des documents par leur contenu, par exemple des emails en spam et non-spam.

Les documents proviennent d'un certain nombre de classes de documents (par exemple, la classe "spam" et la classe "non spam").  

Ces classes de document sont définies comme des ensembles de mots.

La probabilité

$$
{\displaystyle P(w\vert C)}
$$

s'interprète de la manière suivante : sachant que nous sommes dans un document de classe $C$, il s'agit de la probabilité que $w$ soit présent dans ce document.

Une hypothèse forte faite ici est qu'il n'y a aucune dépendance entre la présence de mots. Autrement dit, soient 2 mots quelconques, le fait que l'un des 2 mots soit présent dans un document d'une classe donnée est indépendant du fait que l'autre mot soit présent dans un document appartemant à la même classe.  

On comprend que cette hypothèse est une simplification forte qui diffère de la réalité. Il est facile d'imaginer que la présence d'un mot puisse entraîner une probabilité forte de présence d'un autre mot utilisé fréquemment en association avec le premier. La gestion de la dépendance est très complexe et difficilement quantifiable. Faire l'hypothèse de l'indépendance de la présence des mots est donc plus simple et permet de mener les calculs présentés par la suite.

On considère un document (email) D comme étant un ensemble de mots $\{w_i; 0 \leq i \leq m\}$, $m$ étant le nombre de mots de ce document.

La probabilité :

$$
{\displaystyle P(D\vert C)=\prod _{w_i \in D}P(w_{i}\vert C)}
$$

s'interprète de la manière suivante : sachant que nous sommes dans un document de classe C, il s'agit de la probabilité que tous les mots du document D puissent être présents.

La question qui nous intéresse est la suivante : quelle est la probabilité qu'un document D appartienne à une classe donnée C ?  
C'est à dire : sachant que nous avons un document D, quelle est la probabilité d'être dans C ? Ce qui se traduit par le calcul de $P(C|D)$ ?

Par définition,

$$
{\displaystyle P(D\vert C)={P(D\cap C) \over P(C)}}
$$

et

$$
{\displaystyle P(C\vert D)={P(D\cap C) \over P(D)}}
$$

Le théorème de Bayes nous permet d'inférer la probabilité en termes de vraisemblance.

$$
{\displaystyle P(C\vert D)={P(C) \over P(D)}\,P(D\vert C)}
$$

Si on suppose qu'il n'existe que deux classes mutuellement exclusives, $S$ et $\neg S$ (e.g. spam et non-spam), telles que chaque élément (email) appartienne soit à l'une, soit à l'autre,

$$
{\displaystyle P(D\vert S)=\prod _{w_i \in D}P(w_{i}\vert S)\,}
$$

et

$$
{\displaystyle P(D\vert \neg S)=\prod _{w_i \in D}P(w_{i}\vert \neg S)}
$$

En utilisant le résultat bayésien ci-dessus, on peut écrire :

$$
{\displaystyle P(S\vert D)={P(S) \over P(D)}\,\prod _{w_i \in D}P(w_{i}\vert S)}
$$

$$
{\displaystyle P(\neg S\vert D)={P(\neg S) \over P(D)}\,\prod _{w_i \in D}P(w_{i}\vert \neg S)}
$$

En divisant les deux équations, on obtient :

$$
{\displaystyle {P(S\vert D) \over P(\neg S\vert D)}={P(S)\,\prod _{w_i \in D}P(w_{i}\vert S) \over P(\neg S)\,\prod _{w_i \in D}P(w_{i}\vert \neg S)}}
$$

qui peut être factorisée de nouveau en :

$$
{\displaystyle {P(S\vert D) \over P(\neg S\vert D)}={P(S) \over P(\neg S)}\,\prod _{w_i \in D}{P(w_{i}\vert S) \over P(w_{i}\vert \neg S)}}
$$

Le document peut donc être classifié comme suit : il s'agit de spam si ${\displaystyle {P(S\vert D)\over P(\neg S\vert D)} >1}$, sinon il s'agit d'un courrier normal.

**Remarque**

Il peut être efficace de calculer le logarithme népérien de ces expressions, noté $\ln$. La raison de cette manipulation provient de la propriété des fonctions logarithmes, à savoir que le logarithme d'un produit AB est égal à la somme des logarithmes de A et de B, soit:

$$
\ln(AB) = \ln(A)+\ln(B)
$$

Ce qui va permettre de transformer le produit de la formule précédente en une somme qui est plus facile à calculer à partir des éléments extraits des tableaux.  

Voici les détails des calculs en effectuant cette manipulation :

$$
{\displaystyle \ln\left({P(S\vert D) \over P(\neg S\vert D)}\right)=\ln \left({P(S) \over P(\neg S)}\,\prod _{w_i \in D}{P(w_{i}\vert S) \over P(w_{i}\vert \neg S)}\right)}
$$

$$
{\displaystyle \ln\left({P(S\vert D) \over P(\neg S\vert D)}\right)=\ln \left({P(S) \over P(\neg S)}\right) + \sum _{w_i \in D} \ln \left({P(w_{i}\vert S) \over P(w_{i}\vert \neg S)}\right)}
$$


$$
{\displaystyle
\ln\left({P(S\vert D) \over P(\neg S\vert D)}\right)=
\ln(P(S))-\ln(P(\neg S))+\sum_{w_i \in D}{\left( \ln(P(w_{i}\vert S)-\ln(P(w_{i}\vert \neg S) \right)}
}
$$

Sachant que $\ln 1 = 0$, le test qui permet de détection de spam qui consiste à vérifier que
${\displaystyle {P(S\vert D)\over P(\neg S\vert D)} >1}$
s'écrira ici
${\displaystyle \ln\left({P(S\vert D)\over P(\neg S\vert D)}\right) > 0}$.

Et donc il faut tester si le membre droit de l'égalité est positif.

## Implémentation

Les fichiers `train-features.txt` et `train-labels.txt` servent de données d'apprentissage permettant de calculer les valeurs de ${\displaystyle P(w_{i}\vert S)}$ et de ${\displaystyle P(w_{i} \vert \neg S)}$.

On charge ensuite le fichier `test-features.txt` contenant les 260 emails de test puis on classera chacun de ces mails en spam ou courrier normal à partir de la technique précédente.

On compare ensuite le résultat de la classification avec la classification réelle de ces emails de test que l'on trouve dans le fichier `test-labels.txt`. On représentera pour cela une matrice de confusion permettant de représenter sous forme matricielle le nombre de vrai-positif, faux-positif, vrai-négatif et faux-négatif.

En s'inspirant du code python fourni dans le polycopié "Matrice de confusion", re-écrire le calcul de classification et de représentation de la matrice de confusion en utilisant la méthode `GaussianNB`de la bibliothèque `sklearn.naive_bayes` et comparer les résultats obtenus avec ceux que vous avez calculés précédemment.

**Indications**

A partir des données d'apprentissage, on calcule facilement $P(S)$ comme étant le nombre de $1$ dans le fichier `train-labels.txt` que l'on note $N_S$, divisé par le nombre d'éléments dans ce fichier, noté $N$. De même
${\displaystyle P(\neg S) = N_{\neg S}/N}$ où $N_{\neg S}$ est le nombre de $0$ dans le fichier `train-labels.txt`.

Comment calculer $P(w_i \vert S)$ ?

$w_i$, ième mot d'un document, correspond à un des mots référencé du dictionnaire, par exemple, on suppose que c'est le mot d'indice $j$ du dictionnaire. Il suffit alors de compter dans la matrice contruite précédemment, le nombre de valeurs non nulles dans la colonne $j$ pour tous les mails qui sont considérés comme des spams (c'est à dire `train_labels[i]=1` pour le mail de la ligne $i$). Et on divise ce nombre par $N_S$ pour avoir la probabilité sachant $S$.

Dans cette méthode, le nombre d'occurence des mots dans un mail n'intervient pas (on pourra réfléchir à savoir comment le faire intervenir en extension de cet exercice) et donc on a intérêt à remplacer dans la matrice toutes les valeurs non nulles par des 1. Ainsi on pourra simplifier le calcul de la manière suivante :

$$
{\displaystyle
P(w_i \vert S) = \frac{\sum_{k=1}^{m} \left(M[k,j]\, T[k]\right)}{N_S}
= \frac{\sum_{k=1}^{m} \left(M[k,j]\, T[k]\right)}{\sum_{k=1}^{m} T[k]}
}
$$

où $w_i$ est le mot $j$ du dictionnaire, $m$ est le nombre d'emails dans la base d'apprentissage, $M$ est la matrice `train_matrix`, construite précédemment et $T$ correspond au vecteur `train_labels`.


**Référence**
- [article wikipedia "Classification naïve bayésienne"](https://fr.wikipedia.org/wiki/Classification_na%C3%AFve_bay%C3%A9sienne)
  

In [ ]:
import math
import numpy as np # Ajouté pour les opérations NumPy manquantes dans la version originale
from sklearn.metrics import confusion_matrix # Ajouté car utilisé dans la version originale

# --- Constantes pour le Lissage de Laplace ---
alpha = 1
V = numTokens # Taille du vocabulaire (2500)
# Note: numTrainDocs et numTokens doivent être définis précédemment

# --- Calcul des Probabilités A Priori ---

# Correction: La boucle pour N_no_spam doit aller jusqu'à numTrainDocs (indice 699)
N_spam = 0
for i in range(numTrainDocs):
    if train_labels[i] == 1:
            N_spam += 1
print("N_spam:",N_spam)

N_no_spam = 0
for i in range(numTrainDocs): # Correction de la boucle : numTrainDocs
    if train_labels[i] == 0:
            N_no_spam += 1
print("N_no_spam:",N_no_spam)

def Prob_S(N_spam,numDocs):
  return N_spam/numDocs

def Prob_no_S(N_no_spam,numDocs):
  return N_no_spam/numDocs

# --- Fonctions de Probabilité Conditionnelle (CORRIGÉES) ---

# Calcule P(Wj | S)
def prob_mot_spam(train_matrix, train_labels, j):
  nb_occurence = 0
  # NOTE: C'EST ICI QUE L'ERREUR ÉTAIT (numTrainDocs = 700)
  for i in range(numTrainDocs):
    if train_labels[i] == 1 and train_matrix[i,j] > 0:
      nb_occurence += 1

  # Correction Mathématique: Lissage de Laplace
  return (nb_occurence + alpha) / (N_spam + alpha * V)

# Calcule P(Wj | non S)
def prob_mot_no_spam(train_matrix, train_labels, j):
  nb_occurence = 0
  for i in range(numTrainDocs):
    if train_labels[i] == 0 and train_matrix[i,j] > 0:
      nb_occurence += 1

  # Correction Mathématique: Lissage de Laplace
  return (nb_occurence + alpha) / (N_no_spam + alpha * V)


# --- Fonction Ratio (Forme Produit) ---
def ratio(document_vector, N_spam,numDocs,matrice,labels):

  # Terme a priori: P(S) / P(¬S)
  produit = Prob_S(N_spam,numDocs) / Prob_no_S(N_no_spam,numDocs)

  # Terme de vraisemblance (Produit des ratios conditionnels)
  for j in range(numTokens):

      # Filtrer pour n'inclure que les mots PRÉSENTS
      if document_vector[j] > 0:

          # Les fonctions prob_mot_spam/no_spam appellent ici la matrice de test
          # MAIS elles utilisent numTrainDocs (700) dans leur boucle interne
          P_w_sachant_S = prob_mot_spam(matrice,labels, j)
          P_w_sachant_no_S = prob_mot_no_spam(matrice, labels, j)

          # Multiplier par le ratio du mot j
          produit *= P_w_sachant_S / P_w_sachant_no_S

  return produit

def docIsSpam(ratio_calcule):
# En forme produit, le seuil de décision est 1
  if ratio_calcule > 1:
    print(f"Ratio :{ratio_calcule} > 1")
    print("Le document est un spam")
    return True
  else:
    print(f"Ratio :{ratio_calcule} < 1")
    print("Le document n'est pas un spam")
    return False

# --- Test ---
test_features = np.loadtxt('test-features.txt')
test_labels = np.loadtxt('test-labels.txt')
numTrainDocs_test = 260
numTokens_test = 2500
M_test = np.loadtxt('test-features.txt')
test_matrix = np.zeros((numTrainDocs_test, numTokens_test))
test_labels = np.loadtxt('test-labels.txt')

for i in range(M_test.shape[0]):
    test_matrix[int(M_test[i,0]-1), int(M_test[i,1]-1)] = M_test[i,2]

print("test_matrix :\n",test_matrix)
print("test_labels :\n",test_labels)

N_spam_test = 0
for i in range(numTrainDocs_test):
    if test_labels[i] == 1:
            N_spam_test += 1
print("N_spam_test:",N_spam_test)

N_no_spam_test = 0
for i in range(numTrainDocs_test): #
    if test_labels[i] == 0:
            N_no_spam_test += 1
print("N_no_spam_test:",N_no_spam_test)

for i in range(numTrainDocs_test):
  # CET APPEL EST LA SOURCE DE L'INDEXERROR
  ratio_calcule = ratio(test_matrix[i], N_spam_test,numTrainDocs_test,test_matrix,test_labels)
  docIsSpam(ratio_calcule)

--- RÉSULTATS DE VOTRE MODÈLE NAIVE BAYES (Log-Ratio) ---
Matrice de Confusion:
[[125   5]
 [  0 130]]

Précision (Accuracy) de Votre Modèle : 0.9808
